# Getting started with Amazon Braket program sets
<!--
## Table of Contents

1. [Specifying program sets](#1-specifying-program-sets)
   - [1.1 Specified circuits](#11-specified-circuits)
   - [1.2 Parametrized circuit](#12-parametrized-circuit)
   - [1.3 List of observables](#13-list-of-observables)
2. [Running program sets and accessing results](#2-running-program-sets-and-accessing-the-results)
   - [2.1 Running a program set](#21-running-a-program-set)
   - [2.2 Accessing individual counts](#22-accessing-individual-counts)
   - [2.3 Accessing individual observable expectations](#23-accessing-individual-observable-expectations)
3. [More ways to specify a program set](#3-more-ways-to-specify-a-program-set)
   - [3.1 Sum of observables](#31-sum-of-observables)
   - [3.2 Combining many circuits & many observables (all-to-all)](#32-combining-many-circuits--many-observables-all-to-all)
   - [3.3 Distributing input and observable settings (one-to-one)](#33-distributing-input-and-observable-settings-over-a-list-of-circuits-one-to-one)
4. [Running on a Braket QPU](#4-running-on-a-braket-qpu)
   - [4.1 Submitting a task and obtaining the result](#41-submitting-a-task-and-obtaining-the-result)
   - [4.2 Inspecting the results](#42-inspecting-the-results)
5. [Summary](#summary)
-->

## Overview

Several Braket devices accept a set of programs as input, which are processed together as a `ProgramSet`. While the quantum computational results are the same as if programs in the set were processed as independent Braket `QuantumTask`s, the execution is more efficient and often cheaper. In this notebook we demonstrate various ways of creating program sets and how to submit them to Braket devices and analyze the results.

### What you'll learn:
- How to create and use program sets
- Different ways of combining circuits, parameters, and observables
- How to access and interpret results when you run program sets on simulators and QPUs

### Key Benefits of `ProgramSet`:
- **Efficiency**: Batch processing reduces overhead
- **Cost**: Often cheaper than using individual quantum tasks
- **Organization**: Group related quantum computations together

### Key Terminology

Before diving into the details, let's define the key terms used throughout this notebook:

- **Program Set**: A collection of quantum programs that can be executed together efficiently
- **Executable**: A complete quantum circuit ready for execution (no free parameters, single observable)
- **Entry/Program**: A group of related executables within a program set (e.g., same circuit with different parameters)
- **Circuit Binding**: A program that binds a parametrized circuit to specific parameter values and/or observables
- **Input Sets**: Collections of parameter values for parametrized circuits
- **Observable**: A quantum operator (like Pauli matrices) that defines what to measure
- **Free Parameter**: An unbound variable in a quantum circuit that needs a value assignment

In [1]:
from math import pi

# Utility function for visualizing program sets
from program_set_utils import print_program_set

# Amazon Braket components
from braket.circuits import Circuit, FreeParameter
from braket.circuits.observables import I, X, Y, Z
from braket.devices import LocalSimulator
from braket.program_sets import CircuitBinding, ProgramSet

---
## 1. Specifying program sets

Program sets provide a way to package a list of **circuits**, parameter specification ("**input sets**"), and **observables** into a single bundle of *executables*. An executable, in this context, is a quantum circuit that is ready to be executed and measured (in a specified basis) on a quantum device. This means that there are no free (unbound) parameters in the circuit and we have specified a single (non-composite) observable for the measurement basis. We can have many executables coming from the same underlying circuit (e.g. by changing the parameters of a parametric circuit or changing the observable of a non-parametric one) and they will be grouped together in a single *entry* (or *program*) in the program set. 

![](./images/program_set_legend.png)

### 1.1 Specified circuits
The base constructor of `ProgramSet` allows packaging circuits, which may be completely unrelated to one another, into a single object.

![](./images/program_set_1.png)

In [2]:
program_set_1 = ProgramSet([
    Circuit().h(0).cnot(0,1),
    Circuit().rx(0, pi/4).ry(1, pi/8).cnot(1,0),
    Circuit().t(0).t(1).cz(0,1).s(0).cz(1,2).s(1).s(2),
])

Attributes `total_executables`, `entries`, and the method `to_ir()` can be used to inspect the `ProgramSet` object.

In [3]:
print(f"There are {len(program_set_1)} entries in the program set. First entry is: \n", program_set_1.entries[0])
print(f"The number of total executables in the set is {program_set_1.total_executables}")

There are 3 entries in the program set. First entry is: 
 T  : │  0  │  1  │
      ┌───┐       
q0 : ─┤ H ├───●───
      └───┘   │   
            ┌─┴─┐ 
q1 : ───────┤ X ├─
            └───┘ 
T  : │  0  │  1  │
The number of total executables in the set is 3


In [4]:
program_set_1.total_executables

3

In [5]:
program_set_1.to_ir()

ProgramSet(braketSchemaHeader=BraketSchemaHeader(name='braket.ir.openqasm.program_set', version='1'), programs=[Program(braketSchemaHeader=BraketSchemaHeader(name='braket.ir.openqasm.program', version='1'), source='OPENQASM 3.0;\nbit[2] b;\nqubit[2] q;\nh q[0];\ncnot q[0], q[1];\nb[0] = measure q[0];\nb[1] = measure q[1];', inputs={}), Program(braketSchemaHeader=BraketSchemaHeader(name='braket.ir.openqasm.program', version='1'), source='OPENQASM 3.0;\nbit[2] b;\nqubit[2] q;\nrx(0.7853981633974483) q[0];\nry(0.39269908169872414) q[1];\ncnot q[1], q[0];\nb[0] = measure q[0];\nb[1] = measure q[1];', inputs={}), Program(braketSchemaHeader=BraketSchemaHeader(name='braket.ir.openqasm.program', version='1'), source='OPENQASM 3.0;\nbit[3] b;\nqubit[3] q;\nt q[0];\nt q[1];\ncz q[0], q[1];\ns q[0];\ncz q[1], q[2];\ns q[1];\ns q[2];\nb[0] = measure q[0];\nb[1] = measure q[1];\nb[2] = measure q[2];', inputs={})])

To facilitate visualization of the more complex program sets later in the notebook, we can use the `print_program_set` utility function from the `program_set_utils` module accompanying the notebooks.

In [6]:
print_program_set(program_set_1)

circuit 0
T  : │  0  │  1  │
      ┌───┐       
q0 : ─┤ H ├───●───
      └───┘   │   
            ┌─┴─┐ 
q1 : ───────┤ X ├─
            └───┘ 
T  : │  0  │  1  │
execution 0
--------------------------------------------------------------------------------
circuit 1
T  : │     0      │  1  │
      ┌──────────┐ ┌───┐ 
q0 : ─┤ Rx(0.79) ├─┤ X ├─
      └──────────┘ └─┬─┘ 
      ┌──────────┐   │   
q1 : ─┤ Ry(0.39) ├───●───
      └──────────┘       
T  : │     0      │  1  │
execution 0
--------------------------------------------------------------------------------
circuit 2
T  : │  0  │  1  │  2  │  3  │
      ┌───┐       ┌───┐       
q0 : ─┤ T ├───●───┤ S ├───────
      └───┘   │   └───┘       
      ┌───┐ ┌─┴─┐       ┌───┐ 
q1 : ─┤ T ├─┤ Z ├───●───┤ S ├─
      └───┘ └───┘   │   └───┘ 
                  ┌─┴─┐ ┌───┐ 
q2 : ─────────────┤ Z ├─┤ S ├─
                  └───┘ └───┘ 
T  : │  0  │  1  │  2  │  3  │
execution 0
-----------------------------------------------------------------------

### 1.2 Parametrized circuit
Circuits with free floating-point parameters and a corresponding list of parameter settings can be packaged into a single program set to prepare them for more efficient execution on a QPU. We do that using the `CircuitBinding` class.

![](./images/program_set_2.png)

In [7]:
alpha = FreeParameter('alpha')
beta = FreeParameter('beta')
parametrized_circuit = Circuit().rx(0, alpha).cnot(0,1).ry(1, beta)
print(parametrized_circuit)

T  : │      0      │  1  │     2      │
      ┌───────────┐                    
q0 : ─┤ Rx(alpha) ├───●────────────────
      └───────────┘   │                
                    ┌─┴─┐ ┌──────────┐ 
q1 : ───────────────┤ X ├─┤ Ry(beta) ├─
                    └───┘ └──────────┘ 
T  : │      0      │  1  │     2      │

Unassigned parameters: [alpha, beta].


In [8]:
program_set_2 = ProgramSet(
    CircuitBinding(
        circuit=parametrized_circuit, 
        input_sets={
            'alpha': (0.10, 0.11, 0.22, 0.34, 0.45),
            'beta': (1.01, 1.01, 1.03, 1.04, 1.04),
        })
)
print_program_set(program_set_2)

circuit 0
T  : │      0      │  1  │     2      │
      ┌───────────┐                    
q0 : ─┤ Rx(alpha) ├───●────────────────
      └───────────┘   │                
                    ┌─┴─┐ ┌──────────┐ 
q1 : ───────────────┤ X ├─┤ Ry(beta) ├─
                    └───┘ └──────────┘ 
T  : │      0      │  1  │     2      │

Unassigned parameters: [alpha, beta].
execution 0
	input_set: {'alpha': 0.1, 'beta': 1.01}
execution 1
	input_set: {'alpha': 0.11, 'beta': 1.01}
execution 2
	input_set: {'alpha': 0.22, 'beta': 1.03}
execution 3
	input_set: {'alpha': 0.34, 'beta': 1.04}
execution 4
	input_set: {'alpha': 0.45, 'beta': 1.04}
--------------------------------------------------------------------------------
Total executions: 5


While the total number of executions will be the length of the parameter lists, calling the `to_ir()` method reveals that the parametrized circuit is untouched, and the `CircuitBinding` object connected it to the parameter settings.

In [9]:
program_set_2.to_ir()

ProgramSet(braketSchemaHeader=BraketSchemaHeader(name='braket.ir.openqasm.program_set', version='1'), programs=[Program(braketSchemaHeader=BraketSchemaHeader(name='braket.ir.openqasm.program', version='1'), source='OPENQASM 3.0;\ninput float beta;\ninput float alpha;\nbit[2] b;\nqubit[2] q;\nrx(alpha) q[0];\ncnot q[0], q[1];\nry(beta) q[1];\nb[0] = measure q[0];\nb[1] = measure q[1];', inputs={'alpha': [0.1, 0.11, 0.22, 0.34, 0.45], 'beta': [1.01, 1.01, 1.03, 1.04, 1.04]})])

**Note:** It is also possible to combine fully-specified circuits and circuit bindings by passing them in a single list.

![](./images/program_set_3.png)

In [10]:
program_set_3 = ProgramSet(program_set_1.entries + program_set_2.entries)
print_program_set(program_set_3)

circuit 0
T  : │  0  │  1  │
      ┌───┐       
q0 : ─┤ H ├───●───
      └───┘   │   
            ┌─┴─┐ 
q1 : ───────┤ X ├─
            └───┘ 
T  : │  0  │  1  │
execution 0
--------------------------------------------------------------------------------
circuit 1
T  : │     0      │  1  │
      ┌──────────┐ ┌───┐ 
q0 : ─┤ Rx(0.79) ├─┤ X ├─
      └──────────┘ └─┬─┘ 
      ┌──────────┐   │   
q1 : ─┤ Ry(0.39) ├───●───
      └──────────┘       
T  : │     0      │  1  │
execution 0
--------------------------------------------------------------------------------
circuit 2
T  : │  0  │  1  │  2  │  3  │
      ┌───┐       ┌───┐       
q0 : ─┤ T ├───●───┤ S ├───────
      └───┘   │   └───┘       
      ┌───┐ ┌─┴─┐       ┌───┐ 
q1 : ─┤ T ├─┤ Z ├───●───┤ S ├─
      └───┘ └───┘   │   └───┘ 
                  ┌─┴─┐ ┌───┐ 
q2 : ─────────────┤ Z ├─┤ S ├─
                  └───┘ └───┘ 
T  : │  0  │  1  │  2  │  3  │
execution 0
-----------------------------------------------------------------------

### 1.3 List of observables

When the same circuit needs to be executed multiple times with a different observable at the end, the same `CircuitBinding` object can be used to form the corresponding program set.

![](./images/program_set_4.png)

In [11]:
program_set_4 = ProgramSet(
    CircuitBinding(
        circuit=Circuit().h(0).h(1),
        observables=(
            Z(0) @ Z(1), 
            X(0) @ X(1), 
            Z(0) @ X(1), 
            X(0) @ Z(1),
        )
    )
)
print_program_set(program_set_4)

circuit 0
T  : │  0  │
      ┌───┐ 
q0 : ─┤ H ├─
      └───┘ 
      ┌───┐ 
q1 : ─┤ H ├─
      └───┘ 
T  : │  0  │
execution 0
	observable: z(q[0]) @ z(q[1])
execution 1
	observable: x(q[0]) @ x(q[1])
execution 2
	observable: z(q[0]) @ x(q[1])
execution 3
	observable: x(q[0]) @ z(q[1])
--------------------------------------------------------------------------------
Total executions: 4


Each observable setting is registered as a different executable, identical to how parameter settings were treated. 

The `to_ir()` method reveals how the necessary basis transformations for each observable are automatically implemented as parametrized `rx` and `rz` gates. The name of the corresponding parameters start with `"_OBSERVABLE_"`.

In [12]:
program_set_4.to_ir()

ProgramSet(braketSchemaHeader=BraketSchemaHeader(name='braket.ir.openqasm.program_set', version='1'), programs=[Program(braketSchemaHeader=BraketSchemaHeader(name='braket.ir.openqasm.program', version='1'), source='OPENQASM 3.0;\ninput float _OBSERVABLE_PHI_1;\ninput float _OBSERVABLE_THETA_0;\ninput float _OBSERVABLE_OMEGA_1;\ninput float _OBSERVABLE_PHI_0;\ninput float _OBSERVABLE_THETA_1;\ninput float _OBSERVABLE_OMEGA_0;\nbit[2] b;\nqubit[2] q;\nh q[0];\nh q[1];\nrz(_OBSERVABLE_THETA_0) q[0];\nrx(_OBSERVABLE_PHI_0) q[0];\nrz(_OBSERVABLE_OMEGA_0) q[0];\nrz(_OBSERVABLE_THETA_1) q[1];\nrx(_OBSERVABLE_PHI_1) q[1];\nrz(_OBSERVABLE_OMEGA_1) q[1];\nb[0] = measure q[0];\nb[1] = measure q[1];', inputs={'_OBSERVABLE_THETA_0': [0, 1.5707963267948966, 0, 1.5707963267948966], '_OBSERVABLE_PHI_0': [0, 1.5707963267948966, 0, 1.5707963267948966], '_OBSERVABLE_OMEGA_0': [0, 1.5707963267948966, 0, 1.5707963267948966], '_OBSERVABLE_THETA_1': [0, 1.5707963267948966, 1.5707963267948966, 0], '_OBSERVABL

You can also represent observables in the list as [`PauliString`s](https://amazon-braket-sdk-python.readthedocs.io/en/latest/_apidoc/braket.quantum_information.pauli_string.html#braket.quantum_information.pauli_string.PauliString), or simply strings that produce valid `PauliString`s. Furthermore, you can omit the targets from an observable if the total qubit count of the observable is equal to that of the circuit.

For example, the following program set is equivalent to the one above:

In [13]:
from braket.quantum_information import PauliString

program_set_4 = ProgramSet(
    CircuitBinding(
        circuit=Circuit().h(0).h(1),
        observables=(
            Z(0) @ Z(1), 
            PauliString("XX"),
            "ZX",  # You can also include a phase, just as you would when constructing a PauliString
            X() @ Z(),
        )
    )
)
program_set_4.to_ir()

ProgramSet(braketSchemaHeader=BraketSchemaHeader(name='braket.ir.openqasm.program_set', version='1'), programs=[Program(braketSchemaHeader=BraketSchemaHeader(name='braket.ir.openqasm.program', version='1'), source='OPENQASM 3.0;\ninput float _OBSERVABLE_PHI_1;\ninput float _OBSERVABLE_THETA_0;\ninput float _OBSERVABLE_OMEGA_1;\ninput float _OBSERVABLE_PHI_0;\ninput float _OBSERVABLE_THETA_1;\ninput float _OBSERVABLE_OMEGA_0;\nbit[2] b;\nqubit[2] q;\nh q[0];\nh q[1];\nrz(_OBSERVABLE_THETA_0) q[0];\nrx(_OBSERVABLE_PHI_0) q[0];\nrz(_OBSERVABLE_OMEGA_0) q[0];\nrz(_OBSERVABLE_THETA_1) q[1];\nrx(_OBSERVABLE_PHI_1) q[1];\nrz(_OBSERVABLE_OMEGA_1) q[1];\nb[0] = measure q[0];\nb[1] = measure q[1];', inputs={'_OBSERVABLE_THETA_0': [0, 1.5707963267948966, 0, 1.5707963267948966], '_OBSERVABLE_PHI_0': [0, 1.5707963267948966, 0, 1.5707963267948966], '_OBSERVABLE_OMEGA_0': [0, 1.5707963267948966, 0, 1.5707963267948966], '_OBSERVABLE_THETA_1': [0, 1.5707963267948966, 1.5707963267948966, 0], '_OBSERVABL

A list of observables can be combined with a list of parameter settings inside a single `CircuitBinding` object. The parameter and observable settings are combined all-to-all, and the resulting program set has a large number of executables

![](./images/program_set_5.png)

In [13]:
program_set_5 = ProgramSet(
    CircuitBinding(
        circuit=program_set_2.entries[0].circuit,
        input_sets=program_set_2.entries[0].input_sets,
        observables=program_set_4.entries[0].observables
    )
)
print_program_set(program_set_5)

circuit 0
T  : │      0      │  1  │     2      │
      ┌───────────┐                    
q0 : ─┤ Rx(alpha) ├───●────────────────
      └───────────┘   │                
                    ┌─┴─┐ ┌──────────┐ 
q1 : ───────────────┤ X ├─┤ Ry(beta) ├─
                    └───┘ └──────────┘ 
T  : │      0      │  1  │     2      │

Unassigned parameters: [alpha, beta].
execution 0
	input_set: {'alpha': 0.1, 'beta': 1.01}
	observable: z(q[0]) @ z(q[1])
execution 1
	input_set: {'alpha': 0.1, 'beta': 1.01}
	observable: x(q[0]) @ x(q[1])
execution 2
	input_set: {'alpha': 0.1, 'beta': 1.01}
	observable: z(q[0]) @ x(q[1])
execution 3
	input_set: {'alpha': 0.1, 'beta': 1.01}
	observable: x(q[0]) @ z(q[1])
execution 4
	input_set: {'alpha': 0.11, 'beta': 1.01}
	observable: z(q[0]) @ z(q[1])
execution 5
	input_set: {'alpha': 0.11, 'beta': 1.01}
	observable: x(q[0]) @ x(q[1])
execution 6
	input_set: {'alpha': 0.11, 'beta': 1.01}
	observable: z(q[0]) @ x(q[1])
execution 7
	input_set: {'alpha': 0.11, 

---
# 2. Running program sets and accessing the results

### 2.1 Running a program set

Program sets can be run on Braket's devices in the same way we run quantum tasks - using the `device.run()` method. In this notebook we'll only use `LocalSimulator` as we want to keep the focus on the program set feature. See Section 4 for comments on running on QPU.

In [14]:
device = LocalSimulator()
result_set_1 = device.run(program_set_1, shots=300).result()

⚠️ **Important**: The total number of shots must be evenly divisible by the number of executables in your program set. For example, if you have 3 executables, you can use 300, 603, or 915 shots, but not 500 shots. If we attempt to run a configuration where this is not possible, as is the case in the cell below, where we provide 500 shots for the 3 executables of our `program_set_1`, the attempt will produce an error.

In [15]:
try:
    result = device.run(program_set_1, shots=500).result()
    print(f"SUCCESS: Executed 500 shots divided equally among the {program_set_1.total_executables} executables of the set.")
except ValueError as e:
    print("FAILURE: ", e)

FAILURE:  Total shots must be divisible by number of executables.


An easy way to always ensure that the total shots provided are divisible by the number of executables is to prescribe a specified shot-per-executable count. Here we wish to run 100 shots for each program in the program set, so we pass `shots=program_set.total_executables * 100` to the `run()` method of the device.

In [16]:
shots_per_executable = 100
result_1 = device.run(program_set_1, shots = program_set_1.total_executables * shots_per_executable).result()
result_2 = device.run(program_set_2, shots = program_set_2.total_executables * shots_per_executable).result()
result_3 = device.run(program_set_3, shots = program_set_3.total_executables * shots_per_executable).result()
result_4 = device.run(program_set_4, shots = program_set_4.total_executables * shots_per_executable).result()
result_5 = device.run(program_set_5, shots = program_set_5.total_executables * shots_per_executable).result()

The `ProgramSetQuantumTask` result returned after a successful job completion has different entries corresponding to the entries of the executed program set. Recall the structure of our `program_set_3`, for example, and let us examine `result_3`: 
![](./images/program_set_3.png)

In [17]:
print(f"result_3 has {len(result_3)} composite entries:")
for i in range(len(result_3)):
    print(f"   result_3[{i}] has {len(result_3[i])} measured entries.")

result_3 has 4 composite entries:
   result_3[0] has 1 measured entries.
   result_3[1] has 1 measured entries.
   result_3[2] has 1 measured entries.
   result_3[3] has 5 measured entries.


We may use our `print_program_set` utility function to inspect the results as well.

In [18]:
print_program_set(program_set_3, result_3)

circuit 0
T  : │  0  │  1  │
      ┌───┐       
q0 : ─┤ H ├───●───
      └───┘   │   
            ┌─┴─┐ 
q1 : ───────┤ X ├─
            └───┘ 
T  : │  0  │  1  │
execution 0
	result:  Counter({'00': 57, '11': 43})
--------------------------------------------------------------------------------
circuit 1
T  : │     0      │  1  │
      ┌──────────┐ ┌───┐ 
q0 : ─┤ Rx(0.79) ├─┤ X ├─
      └──────────┘ └─┬─┘ 
      ┌──────────┐   │   
q1 : ─┤ Ry(0.39) ├───●───
      └──────────┘       
T  : │     0      │  1  │
execution 0
	result:  Counter({'00': 93, '10': 5, '11': 1, '01': 1})
--------------------------------------------------------------------------------
circuit 2
T  : │  0  │  1  │  2  │  3  │
      ┌───┐       ┌───┐       
q0 : ─┤ T ├───●───┤ S ├───────
      └───┘   │   └───┘       
      ┌───┐ ┌─┴─┐       ┌───┐ 
q1 : ─┤ T ├─┤ Z ├───●───┤ S ├─
      └───┘ └───┘   │   └───┘ 
                  ┌─┴─┐ ┌───┐ 
q2 : ─────────────┤ Z ├─┤ S ├─
                  └───┘ └───┘ 
T  : │  0  │  1  

### 2.2 Accessing individual counts

To get a particular measurement counter, say, from **circuit 3, execution 2**, we need access it like this:

In [19]:
result_3[3][2].counts

Counter({'00': 77, '01': 23})

To extract all measurement counts, we need to iterate first through programs and second through executions:

In [20]:
for program_result in result_3:
    for execution_result in program_result:
        print(execution_result.counts)

Counter({'00': 57, '11': 43})
Counter({'00': 93, '10': 5, '11': 1, '01': 1})
Counter({'000': 100})
Counter({'00': 79, '01': 21})
Counter({'00': 84, '01': 16})
Counter({'00': 77, '01': 23})
Counter({'00': 74, '01': 25, '11': 1})
Counter({'00': 73, '01': 25, '10': 1, '11': 1})


**Note:** For "product" program sets, i.e. if both `input_sets` and `observables` keywords have been provided to the `CircuitBinding()` or `ProgramSet.product()` constructor (see Section 3), the corresponding result object will list the executions first indexed by `input_sets` and second by `observables`. See the example below.

In [21]:
print_program_set(program_set_5, result_5)

circuit 0
T  : │      0      │  1  │     2      │
      ┌───────────┐                    
q0 : ─┤ Rx(alpha) ├───●────────────────
      └───────────┘   │                
                    ┌─┴─┐ ┌──────────┐ 
q1 : ───────────────┤ X ├─┤ Ry(beta) ├─
                    └───┘ └──────────┘ 
T  : │      0      │  1  │     2      │

Unassigned parameters: [alpha, beta].
execution 0
	input_set: {'alpha': 0.1, 'beta': 1.01}
	observable: z(q[0]) @ z(q[1])
	result:  Counter({'00': 73, '01': 27}), expectation: 0.46
execution 1
	input_set: {'alpha': 0.1, 'beta': 1.01}
	observable: x(q[0]) @ x(q[1])
	result:  Counter({'10': 51, '00': 44, '01': 4, '11': 1}), expectation: -0.1
execution 2
	input_set: {'alpha': 0.1, 'beta': 1.01}
	observable: z(q[0]) @ x(q[1])
	result:  Counter({'00': 90, '01': 9, '11': 1}), expectation: 0.82
execution 3
	input_set: {'alpha': 0.1, 'beta': 1.01}
	observable: x(q[0]) @ z(q[1])
	result:  Counter({'10': 39, '00': 39, '01': 13, '11': 9}), expectation: -0.04
execution 4
	

### 2.3 Accessing individual observable expectations

When observables are part of the program set specification, the expectation values for each execution is also computed.

We can access a particular expectation value, say, from **circuit 0, execution 3** with the following syntax.

In [22]:
result_4[0][3].expectation

np.float64(-0.22)

**Note:** In the measurement-count results above, each bitstring, say $b_1 b_2=$ `"10"`, represents the measured (eigen)state that has an eigenvalue of $(-1)^{b_1}\times(-1)^{b_2}=(-1)\times(+1)=-1$. That is to say (when the individual Paulis of the observable are viewed as multi-qubit operators), an eigenvalue $(-1)^{b_1}=-1$ on the first Pauli of the observable and eigenvalue $(-1)^{b_2}=+1$ on the second Pauli of the observable. Thus, the expectation can be computed from the counts using the following formula.

In [23]:
from collections import Counter


def compute_expectation(counts):
    ev = 0
    for bitstring, count in counts.items():
        ones = Counter(bitstring)["1"]
        ev += (-1)**(ones % 2) * count
    ev /= float(sum(counts.values()))
    return ev

compute_expectation(result_4[0][3].counts)

-0.22

---

## 3. More ways to specify a program set

### 3.1 Sum of observables

Instead of passing a manually constructed list of observables (as in Section 1 above), we may create a composite observable by linearly combining a list of Pauli strings. We do this, for example, when constructing a Hamiltonian for quantum chemistry computations.

In [24]:
hamiltonian = 0.1 * Z(0)@Z(1) + (-5.0) * X(0)@X(1)

The resulting observable can be used to create a program set using the `CircuitBinding` class. Each term in this sum observable will have its own dedicated execution.

![](./images/program_set_6.png)

In [25]:
program_set_6 = ProgramSet(
    CircuitBinding(
        circuit=Circuit().h(0).cnot(0,1),
        observables=hamiltonian,
    )
)
print_program_set(program_set_6)

circuit 0
T  : │  0  │  1  │
      ┌───┐       
q0 : ─┤ H ├───●───
      └───┘   │   
            ┌─┴─┐ 
q1 : ───────┤ X ├─
            └───┘ 
T  : │  0  │  1  │
execution 0
	observable: 0.1 * z(q[0]) @ z(q[1])
execution 1
	observable: -5.0 * x(q[0]) @ x(q[1])
sum observable: 0.1 * z(q[0]) @ z(q[1]) - 5.0 * x(q[0]) @ x(q[1])

--------------------------------------------------------------------------------
Total executions: 2


**Note** Once run, the corresponding program set result object facilitates computing the expectation value of the sum observable with `expectation()` method of the program result. Individual observable expectations can be accessed the `expectation` field of the individual executables.

In [26]:
result_set_6 = device.run(program_set_6, shots=program_set_6.total_executables*100).result()
print(f"Hamitlonian expectation value: {result_set_6[0].expectation():.2f}")
print(f"Individual term expectations: {result_set_6[0][0].expectation:.2f}, {result_set_6[0][1].expectation:.2f}")

Hamitlonian expectation value: -4.90
Individual term expectations: 0.10, -5.00


**Note** When a sum observable is provided to a program with many inputs, an all-to-all mapping is done akin to our `problem_set_5` example. They will still be in a single program entry in the program set. In this case, we can access the expectation values of the sum Hamiltonian on the individual states by providing an index to the `CompositeEntry.expectation(...)` call.

In [27]:
program_set_6a = ProgramSet(
    CircuitBinding(
        circuit=program_set_2.entries[0].circuit,
        input_sets=program_set_2.entries[0].input_sets,
        observables=hamiltonian
    )
)
print_program_set(program_set_6a)

circuit 0
T  : │      0      │  1  │     2      │
      ┌───────────┐                    
q0 : ─┤ Rx(alpha) ├───●────────────────
      └───────────┘   │                
                    ┌─┴─┐ ┌──────────┐ 
q1 : ───────────────┤ X ├─┤ Ry(beta) ├─
                    └───┘ └──────────┘ 
T  : │      0      │  1  │     2      │

Unassigned parameters: [alpha, beta].
execution 0
	input_set: {'alpha': 0.1, 'beta': 1.01}
	observable: 0.1 * z(q[0]) @ z(q[1])
execution 1
	input_set: {'alpha': 0.1, 'beta': 1.01}
	observable: -5.0 * x(q[0]) @ x(q[1])
execution 2
	input_set: {'alpha': 0.11, 'beta': 1.01}
	observable: 0.1 * z(q[0]) @ z(q[1])
execution 3
	input_set: {'alpha': 0.11, 'beta': 1.01}
	observable: -5.0 * x(q[0]) @ x(q[1])
execution 4
	input_set: {'alpha': 0.22, 'beta': 1.03}
	observable: 0.1 * z(q[0]) @ z(q[1])
execution 5
	input_set: {'alpha': 0.22, 'beta': 1.03}
	observable: -5.0 * x(q[0]) @ x(q[1])
execution 6
	input_set: {'alpha': 0.34, 'beta': 1.04}
	observable: 0.1 * z(q[0]) @ z

In [28]:
result_set_6a = device.run(program_set_6a, shots=program_set_6a.total_executables*100).result()

print(f"Hamiltonian expectation value for the third (idx=2) input state: {result_set_6a[0].expectation(2):.2f}")
print(f"Individual term expectations for the third (idx=2) input state: {result_set_6a[0][4].expectation:.2f}, {result_set_6a[0][5].expectation:.2f}")

Hamiltonian expectation value for the third (idx=2) input state: 0.16
Individual term expectations for the third (idx=2) input state: 0.06, 0.10


### 3.2 Combining many circuits & many observables (all-to-all)

A common pattern of quantum computing is needing to evaluate a large number of observables. When this is needed on a single (parametrized) circuit, `CircuitBinding` can be used directly to construct the program set (see section 1.3).

To prescribe the evaluation of many observables on many circuits in an all-to-all fashion, one can use the `.product()` method of the `ProgramSet` class.

![](./images/program_set_7.png)

In [29]:
program_set_7 = ProgramSet.product(
    circuits=[
        Circuit().h(0).cnot(0,1),
        Circuit().rx(0, pi/4).ry(1, pi/8).cnot(1,0),
        CircuitBinding(
            circuit=parametrized_circuit, 
            input_sets={
                'alpha': (0.10, 0.11, 0.22, 0.34, 0.45),
                'beta': (1.01, 1.01, 1.03, 1.04, 1.04),
            })
    ],
    observables=(
        Z(0) @ Z(1), 
        X(0) @ X(1), 
        Z(0) @ X(1), 
        X(0) @ Z(1),
    )
)
print_program_set(program_set_7)

circuit 0
T  : │  0  │  1  │
      ┌───┐       
q0 : ─┤ H ├───●───
      └───┘   │   
            ┌─┴─┐ 
q1 : ───────┤ X ├─
            └───┘ 
T  : │  0  │  1  │
execution 0
	observable: z(q[0]) @ z(q[1])
execution 1
	observable: x(q[0]) @ x(q[1])
execution 2
	observable: z(q[0]) @ x(q[1])
execution 3
	observable: x(q[0]) @ z(q[1])
--------------------------------------------------------------------------------
circuit 1
T  : │     0      │  1  │
      ┌──────────┐ ┌───┐ 
q0 : ─┤ Rx(0.79) ├─┤ X ├─
      └──────────┘ └─┬─┘ 
      ┌──────────┐   │   
q1 : ─┤ Ry(0.39) ├───●───
      └──────────┘       
T  : │     0      │  1  │
execution 0
	observable: z(q[0]) @ z(q[1])
execution 1
	observable: x(q[0]) @ x(q[1])
execution 2
	observable: z(q[0]) @ x(q[1])
execution 3
	observable: x(q[0]) @ z(q[1])
--------------------------------------------------------------------------------
circuit 2
T  : │      0      │  1  │     2      │
      ┌───────────┐                    
q0 : ─┤ Rx(alpha) ├───●─

### 3.3 Distributing input and observable settings over a list of circuits (one-to-one)

**Option 1:** Using its `.zip()` method, a program set can be created by mixing different circuits, input sets, and observables. The lengths of all three need to match, and the resulting program set will have the that number of executables.

![](./images/program_set_8.png)

In [30]:
program_set_8 = ProgramSet.zip(
    circuits=[
        Circuit().h(0).cnot(0,1),
        Circuit().rx(0, pi/4).ry(1, pi/8).cnot(1,0),
        Circuit().rx(0, alpha).cnot(0,1).ry(1, beta),
        Circuit().rx(0, alpha).cnot(0,1).ry(1, beta),
        Circuit().rx(0, alpha).cnot(0,1).ry(1, beta),
        Circuit().rx(0, alpha).cnot(0,1).ry(1, beta),
        Circuit().rx(0, alpha).cnot(0,1).ry(1, beta),
    ],
    input_sets=[
        {},
        {},
        {'alpha': 0.10, 'beta': 1.01},
        {'alpha': 0.11, 'beta': 1.01},
        {'alpha': 0.22, 'beta': 1.03},
        {'alpha': 0.34, 'beta': 1.04},
        {'alpha': 0.45, 'beta': 1.04},
    ],
    observables=[
        Z(0) @ Z(1), 
        Z(0) @ Z(1), 
        Z(0) @ Z(1), 
        X(0) @ X(1), 
        Z(0) @ X(1), 
        X(0) @ Z(1),
        I(0) @ Y(1),
        
    ]
)

print_program_set(program_set_8)

circuit 0
T  : │  0  │  1  │
      ┌───┐       
q0 : ─┤ H ├───●───
      └───┘   │   
            ┌─┴─┐ 
q1 : ───────┤ X ├─
            └───┘ 
T  : │  0  │  1  │
execution 0
	observable: z(q[0]) @ z(q[1])
--------------------------------------------------------------------------------
circuit 1
T  : │     0      │  1  │
      ┌──────────┐ ┌───┐ 
q0 : ─┤ Rx(0.79) ├─┤ X ├─
      └──────────┘ └─┬─┘ 
      ┌──────────┐   │   
q1 : ─┤ Ry(0.39) ├───●───
      └──────────┘       
T  : │     0      │  1  │
execution 0
	observable: z(q[0]) @ z(q[1])
--------------------------------------------------------------------------------
circuit 2
T  : │      0      │  1  │     2      │
      ┌───────────┐                    
q0 : ─┤ Rx(alpha) ├───●────────────────
      └───────────┘   │                
                    ┌─┴─┐ ┌──────────┐ 
q1 : ───────────────┤ X ├─┤ Ry(beta) ├─
                    └───┘ └──────────┘ 
T  : │      0      │  1  │     2      │

Unassigned parameters: [alpha, beta].
exe

**Option 2:** Using the `.zip()` method, a single `CircuitBinding` object (which represents a single parametrized circuit and its parameter values) can be mapped to a list of observables. Similar to Option 1 above, this one-to-one mapping requires that the number of parameter settings in the `CircuitBinding` object is the same and the number of observables, and the resulting program set will have that number of executions.

![](./images/program_set_9.png)

In [31]:
program_set_9 = ProgramSet.zip(
    circuits=CircuitBinding(
        circuit=parametrized_circuit, 
        input_sets={
            'alpha': (0.10, 0.11, 0.22, 0.34, 0.45),
            'beta': (1.01, 1.01, 1.03, 1.04, 1.04),
        }
    ),
    observables=[
        Z(0) @ Z(1), 
        X(0) @ X(1), 
        Z(0) @ X(1), 
        X(0) @ Z(1),
        I(0) @ Y(1)
    ]
)
print_program_set(program_set_9)

circuit 0
T  : │      0      │  1  │     2      │
      ┌───────────┐                    
q0 : ─┤ Rx(alpha) ├───●────────────────
      └───────────┘   │                
                    ┌─┴─┐ ┌──────────┐ 
q1 : ───────────────┤ X ├─┤ Ry(beta) ├─
                    └───┘ └──────────┘ 
T  : │      0      │  1  │     2      │

Unassigned parameters: [alpha, beta].
execution 0
	input_set: {'alpha': 0.1, 'beta': 1.01}
	observable: z(q[0]) @ z(q[1])
--------------------------------------------------------------------------------
circuit 1
T  : │      0      │  1  │     2      │
      ┌───────────┐                    
q0 : ─┤ Rx(alpha) ├───●────────────────
      └───────────┘   │                
                    ┌─┴─┐ ┌──────────┐ 
q1 : ───────────────┤ X ├─┤ Ry(beta) ├─
                    └───┘ └──────────┘ 
T  : │      0      │  1  │     2      │

Unassigned parameters: [alpha, beta].
execution 0
	input_set: {'alpha': 0.11, 'beta': 1.01}
	observable: x(q[0]) @ x(q[1])
------------

---
## 4. Running on a Braket QPU

### 4.1. Submitting a task and obtaining the result

We can run program sets on Braket devices the same way we used the `LocalSimulator`.

In [ ]:
# from braket.aws import AwsDevice
# from braket.devices import Devices
# 
# device = AwsDevice(Devices.IQM.Garnet)
# device = AwsDevice(Devices.IQM.Emerald)
# device = AwsDevice(Devices.Rigetti.Ankaa3)
# device = LocalSimulator() 

# uncomment the imports and device above to select a QPU 

aws_task_2 = device.run(program_set_2, shots=program_set_2.total_executables * 100)
print(aws_task_2.id)

7d1f1d5f-95d2-41ce-8767-c1edf14e6d95


These tasks stand in queue, and will complete after some time. We can pick up the thread from here using the quantum task ARN to instantiate the `AwsQuantumTask` object once again.
```python
from braket.aws import AwsQuantumTask

# Below we specify the region and AWS account number where the tasks are run
REGION = 'us-east-1'
ACCOUNT_ID = '000000000000' #<-- Replace with your 12-digit AWS account number

aws_task_2 = AwsQuantumTask(f"arn:aws:braket:{REGION}:{ACCOUNT_ID}:quantum-task/{aws_task_2.id}"
```

Periodically checking their status will allow us to see when they are all completed.

In [33]:
aws_task_2.state()

'COMPLETED'

Once all tasks are completed, we retrieve their results.

In [34]:
aws_result_2 = aws_task_2.result()

### 4.2 Inspecting the results

In Section 2 we discussed the structure of the `ProgramSetQuantumTaskResult` object containing the results after running a program set and showed how to obtain measurement counts and expectation values. Here we will highlight some other fields of the result instance that help analyzing the task outcome.

Among other things, the **metadata** in a program set result contains information about the total number of shots.

In [ ]:
aws_result_2.task_metadata.dict()

{'braketSchemaHeader': {'name': 'braket.task_result.program_set_task_metadata',
  'version': '1'},
 'id': '7d1f1d5f-95d2-41ce-8767-c1edf14e6d95',
 'deviceId': 'braket_sv',
 'requestedShots': 500,
 'successfulShots': 500,
 'programMetadata': [{'executables': [{}, {}, {}, {}, {}]}],
 'deviceParameters': None,
 'createdAt': None,
 'endedAt': None,
 'status': None,
 'totalFailedExecutables': 0}

If the number of successful shots is lower than the requested number of shots, it will be reflected here.

In [ ]:
aws_result_2.task_metadata.requestedShots, aws_result_2.task_metadata.successfulShots

(500, 500)

If the number of successful shots is lower than the requested number of shots, it will be reflected here.

In [ ]:
aws_result_2.task_metadata.totalFailedExecutables

0

The **programs** executed in the tasks are also returned as part of the result.

In [38]:
for program in aws_result_2.programs:
    print(program)

braketSchemaHeader=BraketSchemaHeader(name='braket.ir.openqasm.program', version='1') source='OPENQASM 3.0;\ninput float beta;\ninput float alpha;\nbit[2] b;\nqubit[2] q;\nrx(alpha) q[0];\ncnot q[0], q[1];\nry(beta) q[1];\nb[0] = measure q[0];\nb[1] = measure q[1];' inputs={'alpha': [0.1, 0.11, 0.22, 0.34, 0.45], 'beta': [1.01, 1.01, 1.03, 1.04, 1.04]}


Finally, **additional metadata** for each program is available in the specific `CompositeEntry` result for each program,

In [39]:
aws_result_2[0].additional_metadata.dict()

{'action': None,
 'dwaveMetadata': None,
 'ionqMetadata': None,
 'rigettiMetadata': None,
 'oqcMetadata': None,
 'xanaduMetadata': None,
 'queraMetadata': None,
 'simulatorMetadata': None,
 'iqmMetadata': None}

---

## Summary

In this notebook, we've explored the `ProgramSet` feature of Amazon Braket, which enables efficient batch processing of quantum computations. Here's what we covered:

### Key Concepts Learned:
1. **Program Set Basics**: How to package multiple quantum circuits into a single execution unit
2. **Circuit Binding**: Binding parametrized circuits to specific parameter values and observables
3. **Combination Patterns**: Different ways to combine circuits, parameters, and observables (`product()`, `zip()`)
4. **Result Handling**: How to access and interpret results from program set executions

### Main Construction Methods:
- **Direct Construction**: `ProgramSet([circuit1, circuit_binding2, ...])`
- **Parametrized Circuits**: `ProgramSet(CircuitBinding(circuit, input_sets={...}))`
- **Different Observables**: `ProgramSet(CircuitBinding(circuit, observables=[...]))`
- **Product Combinations**:`ProgramSet.product(circuits=..., observables=...)`
- **Zip Combinations**:
    - `ProgramSet.zip(circuits=..., input_sets=..., observables=...)`
    - `ProgramSet.zip(CircuitBinding(circuit, input_sets={...}), observables=...)`

`ProgramSet` is a powerful tool for scaling quantum computations efficiently, making it ideal for variational algorithms, parameter sweeps, and multiple observable measurements in quantum applications.